# **VGG16 Using CNN**

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import datetime

print("TensorFlow version:", tf.__version__)

class VGG16FoodClassifier:
    def __init__(self):
        self.data_dir = '/kaggle/input/food-classification-34-classes/food34_200_per_class'
        self.img_size = (224, 224)
        self.batch_size = 32
        self.epochs = 50
        self.num_classes = 34
        self.model_path = '/kaggle/working/vgg16_food_model.h5'
        self.report_path = '/kaggle/working/VGG16_Model.txt'

    def load_data(self):
        """Load and prepare data"""
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            brightness_range=[0.8, 1.2],
            fill_mode='nearest'
        )

        val_datagen = ImageDataGenerator(rescale=1./255)

        train_dir = os.path.join(self.data_dir, 'train')
        val_dir = os.path.join(self.data_dir, 'val')

        train_gen = train_datagen.flow_from_directory(
            train_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=True
        )

        val_gen = val_datagen.flow_from_directory(
            val_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=False
        )

        print(f"Found {train_gen.samples} training images")
        print(f"Found {val_gen.samples} validation images")
        return train_gen, val_gen, train_gen.class_indices

    def build_model(self):
        """Build VGG16 model with ImageNet weights"""
        print("Building VGG16 with ImageNet weights...")
        
        # Load pre-trained VGG16
        base_model = VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )

        # Freeze base model initially
        base_model.trainable = False

        # Add custom classification head
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.3)(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
        )
        
        print("VGG16 model built successfully with ImageNet weights!")
        return model

    def train_model(self, train_generator, val_generator):
        """Train the model with two-phase approach"""
        model = self.build_model()

        callbacks = [
            EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
            ModelCheckpoint(self.model_path, monitor='val_accuracy', save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7)
        ]

        print("Phase 1: Training with frozen base layers...")
        history1 = model.fit(
            train_generator,
            epochs=30,
            validation_data=val_generator,
            callbacks=callbacks,
            verbose=1
        )

        print("Phase 2: Fine-tuning with unfrozen layers...")
        # Unfreeze some layers for fine-tuning
        for layer in model.layers[-8:]:
            if not isinstance(layer, BatchNormalization):
                layer.trainable = True

        # Recompile with lower learning rate
        model.compile(
            optimizer=Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
        )

        history2 = model.fit(
            train_generator,
            epochs=self.epochs,
            validation_data=val_generator,
            callbacks=callbacks,
            verbose=1,
            initial_epoch=30
        )
        
        return model, history2

    def calculate_detailed_metrics(self, y_true, y_pred, num_classes):
        """Calculate TP, TN, FP, FN for each class"""
        cm = confusion_matrix(y_true, y_pred)
        
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - TP
        FN = np.sum(cm, axis=1) - TP
        TN = np.sum(cm) - (FP + FN + TP)
        
        return TP, TN, FP, FN, cm

    def generate_report(self, model, val_generator, class_indices):
        """Generate comprehensive validation report with all metrics"""
        print("Generating detailed validation report...")
        
        # Get predictions
        y_pred = model.predict(val_generator)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = val_generator.classes
        
        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred_classes)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='weighted')
        
        # Calculate TP, TN, FP, FN
        TP, TN, FP, FN, cm = self.calculate_detailed_metrics(y_true, y_pred_classes, self.num_classes)
        
        # Generate report content
        report_content = f"""
VGG16 MODEL - COMPREHENSIVE VALIDATION REPORT
============================================================
Generated: {datetime.datetime.now()}

DATASET INFORMATION:
- Training samples: {val_generator.samples}
- Number of classes: {self.num_classes}
- Class names: {list(class_indices.keys())}

OVERALL METRICS:
----------------------------------------
Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)
Precision: {precision:.4f} ({precision*100:.2f}%)
Recall:    {recall:.4f} ({recall*100:.2f}%)
F1-Score:  {f1:.4f} ({f1*100:.2f}%)

CONFUSION MATRIX SUMMARY:
----------------------------------------
Total True Positives (TP):  {int(np.sum(TP))}
Total True Negatives (TN):  {int(np.sum(TN))}
Total False Positives (FP): {int(np.sum(FP))}
Total False Negatives (FN): {int(np.sum(FN))}

DETAILED PER-CLASS METRICS:
----------------------------------------
{'Class':<20} {'TP':<8} {'TN':<8} {'FP':<8} {'FN':<8} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}
{'-'*90}
"""
        # Add per-class metrics
        class_names = list(class_indices.keys())
        for i, class_name in enumerate(class_names):
            class_precision = TP[i] / (TP[i] + FP[i]) if (TP[i] + FP[i]) > 0 else 0
            class_recall = TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0
            class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0
            
            report_content += f"{class_name:<20} {int(TP[i]):<8} {int(TN[i]):<8} {int(FP[i]):<8} {int(FN[i]):<8} {class_precision:.4f}     {class_recall:.4f}      {class_f1:.4f}\n"

        # Performance assessment
        report_content += f"""
PERFORMANCE ASSESSMENT:
----------------------------------------
"""
        if accuracy >= 0.90:
            report_content += "90%+ Accuracy Achieved!"
        elif accuracy >= 0.80:
            report_content += "80%+ Accuracy!"
        elif accuracy >= 0.70:
            report_content += "70%+ Accuracy!"
        elif accuracy >= 0.60:
            report_content += "Needs Improvement"
        else:
            report_content += "Model needs significant improvement"

        report_content += f"""

MODEL TRAINING INFORMATION:
----------------------------------------
- Pre-trained model: VGG16 with ImageNet weights
- Total epochs: {self.epochs}
- Batch size: {self.batch_size}
- Image size: {self.img_size}
- Number of classes: {self.num_classes}
- Model saved: {self.model_path}
"""

        # Save report
        with open(self.report_path, 'w') as f:
            f.write(report_content)

        print(f"Report saved to: {self.report_path}")
        return report_content

    def run(self):
        """Main execution function"""
        print("STARTING VGG16 FOOD CLASSIFICATION")
        print("=" * 60)
        
        # Load data
        train_gen, val_gen, class_indices = self.load_data()
        
        # Train model
        model, history = self.train_model(train_gen, val_gen)
        print("Training completed!")
        
        # Generate report
        report_content = self.generate_report(model, val_gen, class_indices)
        
        # Final results summary
        print(f"\nFINAL RESULTS SUMMARY:")
        print(f"   Accuracy:  {history.history['val_accuracy'][-1]:.4f} ({history.history['val_accuracy'][-1]*100:.2f}%)")
        print(f"   Precision: {history.history['val_precision'][-1]:.4f} ({history.history['val_precision'][-1]*100:.2f}%)")
        print(f"   Recall:    {history.history['val_recall'][-1]:.4f} ({history.history['val_recall'][-1]*100:.2f}%)")
        print(f"   Model saved: {self.model_path}")
        print(f"   Report saved: {self.report_path}")
        
        return model, report_content

# Run VGG16 Model
print("Starting VGG16 Model...")
vgg16_classifier = VGG16FoodClassifier()
vgg16_model, vgg16_report = vgg16_classifier.run()


2025-10-22 02:45:02.948335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761101103.192788      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761101103.262320      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.18.0
Starting VGG16 Model...
STARTING VGG16 FOOD CLASSIFICATION
Found 7956 images belonging to 34 classes.
Found 1372 images belonging to 34 classes.
Found 7956 training images
Found 1372 validation images
Building VGG16 with ImageNet weights...


I0000 00:00:1761101122.191808      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1761101122.192567      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
VGG16 model built successfully with ImageNet weights!
Phase 1: Training with frozen base layers...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1761101129.615176     116 service.cc:148] XLA service 0x7f3768010850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761101129.616247     116 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1761101129.616283     116 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1761101130.106089     116 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/249 ━━━━━━━━━━━━━━━━━━━━ 1:06:04 16s/step - accuracy: 0.0625 - loss: 3.7005 - precision: 0.0000e+00 - recall: 0.0000e+00

I0000 00:00:1761101141.447932     116 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


249/249 ━━━━━━━━━━━━━━━━━━━━ 253s 954ms/step - accuracy: 0.0607 - loss: 3.4779 - precision: 0.2817 - recall: 3.6258e-04 - val_accuracy: 0.2981 - val_loss: 2.6570 - val_precision: 0.2222 - val_recall: 0.0015 - learning_rate: 0.0010
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 162s 651ms/step - accuracy: 0.2082 - loss: 2.7753 - precision: 0.6132 - recall: 0.0206 - val_accuracy: 0.4162 - val_loss: 2.0814 - val_precision: 0.7857 - val_recall: 0.1042 - learning_rate: 0.0010
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 163s 654ms/step - accuracy: 0.3103 - loss: 2.3918 - precision: 0.6946 - recall: 0.0816 - val_accuracy: 0.4483 - val_loss: 1.9395 - val_precision: 0.7525 - val_recall: 0.1640 - learning_rate: 0.0010
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 161s 645ms/step - accuracy: 0.3567 - loss: 2.2263 - precision: 0.6911 - recall: 0.1217 - val_accuracy: 0.4832 - val_loss: 1.7967 - val_precision: 0.7591 - val_recall: 0.2573 - learning_rate: 0.0010
Epoch 5/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 163s 656ms/step

# **Custom Model Usinng CNN**

In [ ]:
# Check if GPU is available and configure for maximum performance
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

# Configure GPU for better performance
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✓ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
import logging
import sys
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import datetime
import json
from pathlib import Path

print("TensorFlow version:", tf.__version__)

# **Configuration Class**
class CustomModelConfig:
    """Configuration class for Custom CNN model parameters and paths"""
    def __init__(self):
        # Use appropriate path for your environment
        self.DATA_DIR = '/kaggle/input/food-classification-34-classes/food34_200_per_class'  # For Kaggle
        
        self.IMG_SIZE = (224, 224)
        self.BATCH_SIZE = 32
        self.EPOCHS = 50  
        self.NUM_CLASSES = 34
        
        # Save paths
        self.CUSTOM_MODEL_PATH = '/kaggle/working/custom_cnn_food_model.h5'
        self.REPORT_PATH = '/kaggle/working/Custom_CNN_Model_Report.txt'
        self.LOG_PATH = '/kaggle/working/custom_cnn_training.log'

        # Data augmentation parameters (more aggressive for custom model)
        self.ROTATION_RANGE = 25
        self.WIDTH_SHIFT_RANGE = 0.15
        self.HEIGHT_SHIFT_RANGE = 0.15
        self.SHEAR_RANGE = 0.15
        self.ZOOM_RANGE = 0.15
        self.HORIZONTAL_FLIP = True
        self.VERTICAL_FLIP = True
        self.BRIGHTNESS_RANGE = [0.8, 1.2]
        self.VALIDATION_SPLIT = 0.2

# **Setup Logging Function**
def setup_logging_custom(config):
    """Setup logging for Custom CNN"""
    try:
        # Create logger
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)

        # Clear any existing handlers
        for handler in logger.handlers[:]:
            logger.removeHandler(handler)

        # Create formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

        # File handler
        file_handler = logging.FileHandler(config.LOG_PATH, mode='w')
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)

        # Stream handler (console)
        stream_handler = logging.StreamHandler(sys.stdout)
        stream_handler.setLevel(logging.INFO)
        stream_handler.setFormatter(formatter)

        # Add handlers to logger
        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)

        logging.info("Custom CNN Logging setup successfully for Kaggle!")
        logging.info(f"Log file: {config.LOG_PATH}")

        return True
    except Exception as e:
        print(f"Custom CNN Logging setup failed: {e}")
        return False

# **Data Generator Class**
class CustomDataGenerator:
    """Handles data loading and augmentation for Custom CNN"""

    def __init__(self, config):
        self.config = config
        self.train_generator = None
        self.validation_generator = None
        self.class_indices = None
        logging.info("CustomDataGenerator initialized for Kaggle")

    def create_data_generators(self):
        """Create data generators with augmentation"""
        try:
            logging.info("Creating Custom CNN data generators with augmentation...")

            # Training generator with more aggressive augmentation for custom model
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=self.config.ROTATION_RANGE,
                width_shift_range=self.config.WIDTH_SHIFT_RANGE,
                height_shift_range=self.config.HEIGHT_SHIFT_RANGE,
                shear_range=self.config.SHEAR_RANGE,
                zoom_range=self.config.ZOOM_RANGE,
                horizontal_flip=self.config.HORIZONTAL_FLIP,
                vertical_flip=self.config.VERTICAL_FLIP,
                brightness_range=self.config.BRIGHTNESS_RANGE,
                fill_mode='nearest'
            )

            # Validation generator (only rescaling)
            val_datagen = ImageDataGenerator(rescale=1./255)

            # Create generators
            train_dir = os.path.join(self.config.DATA_DIR, 'train')
            val_dir = os.path.join(self.config.DATA_DIR, 'val')

            self.train_generator = train_datagen.flow_from_directory(
                train_dir,
                target_size=self.config.IMG_SIZE,
                batch_size=self.config.BATCH_SIZE,
                class_mode='categorical',
                shuffle=True,
                seed=42
            )

            self.validation_generator = val_datagen.flow_from_directory(
                val_dir,
                target_size=self.config.IMG_SIZE,
                batch_size=self.config.BATCH_SIZE,
                class_mode='categorical',
                shuffle=False,
                seed=42
            )

            self.class_indices = self.train_generator.class_indices
            logging.info(f"Custom CNN - Found {len(self.class_indices)} classes")
            logging.info(f"Custom CNN - Training samples: {self.train_generator.samples}")
            logging.info(f"Custom CNN - Validation samples: {self.validation_generator.samples}")

            return True

        except Exception as e:
            logging.error(f"Error creating Custom CNN data generators: {str(e)}")
            raise

# **Custom CNN Model Class**
class CustomFoodModel:
    """Builds and manages the custom CNN model from scratch"""

    def __init__(self, config, num_classes):
        self.config = config
        self.num_classes = num_classes
        self.model = None
        self.history = None
        logging.info("CustomFoodModel initialized for Kaggle")

    def build_model(self):
        """Build custom CNN model from scratch with optimized architecture"""
        try:
            logging.info("Building Custom CNN model from scratch...")

            self.model = Sequential([
                # First Conv Block
                Conv2D(64, (3, 3), activation='relu', padding='same',
                       input_shape=(*self.config.IMG_SIZE, 3)),
                BatchNormalization(),
                Conv2D(64, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                MaxPooling2D(2, 2),
                Dropout(0.25),

                # Second Conv Block
                Conv2D(128, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                Conv2D(128, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                MaxPooling2D(2, 2),
                Dropout(0.25),

                # Third Conv Block
                Conv2D(256, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                Conv2D(256, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                MaxPooling2D(2, 2),
                Dropout(0.25),

                # Fourth Conv Block
                Conv2D(512, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                Conv2D(512, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                MaxPooling2D(2, 2),
                Dropout(0.25),

                # Fifth Conv Block
                Conv2D(512, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                Conv2D(512, (3, 3), activation='relu', padding='same'),
                BatchNormalization(),
                MaxPooling2D(2, 2),
                Dropout(0.3),

                # Classifier Head
                Flatten(),
                Dense(1024, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(512, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(256, activation='relu'),
                BatchNormalization(),
                Dropout(0.3),
                Dense(self.num_classes, activation='softmax')
            ])

            logging.info("Custom CNN model built successfully!")
            return True

        except Exception as e:
            logging.error(f"Error building Custom CNN model: {str(e)}")
            # Fallback: Simpler model if memory issues
            logging.info("Creating simpler fallback model...")
            self._create_simpler_model()
            return True

    def _create_simpler_model(self):
        """Create a simpler CNN model as fallback"""
        self.model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(*self.config.IMG_SIZE, 3)),
            MaxPooling2D(2, 2),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D(2, 2),
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(256, activation='relu'),
            Dropout(0.3),
            Dense(self.num_classes, activation='softmax')
        ])
        logging.info("Simpler fallback model created successfully!")

    def compile_model(self):
        """Compile the Custom CNN model with appropriate settings"""
        try:
            logging.info("Compiling Custom CNN model...")

            self.model.compile(
                optimizer=Adam(learning_rate=0.001),  # Higher learning rate for custom model
                loss='categorical_crossentropy',
                metrics=[
                    'accuracy',
                    Precision(name='precision'),
                    Recall(name='recall')
                ]
            )

            logging.info("Custom CNN model compiled successfully!")
            return True

        except Exception as e:
            logging.error(f"Error compiling Custom CNN model: {str(e)}")
            raise

    def get_callbacks(self):
        """Define training callbacks optimized for custom model"""
        try:
            callbacks = [
                EarlyStopping(
                    monitor='val_accuracy',
                    patience=12,  # More patience for custom model
                    restore_best_weights=True,
                    verbose=1
                ),
                ModelCheckpoint(
                    self.config.CUSTOM_MODEL_PATH,
                    monitor='val_accuracy',
                    save_best_only=True,
                    save_weights_only=False,
                    verbose=1
                ),
                ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,  # Less aggressive reduction
                    patience=6,
                    min_lr=1e-6,
                    verbose=1
                )
            ]
            logging.info("Custom CNN callbacks defined successfully!")
            return callbacks
        except Exception as e:
            logging.error(f"Error creating Custom CNN callbacks: {str(e)}")
            raise

    def display_summary(self):
        """Display model summary"""
        try:
            self.model.summary()
            return True
        except Exception as e:
            logging.error(f"Error displaying Custom CNN model summary: {str(e)}")
            raise

# **Training Class**
class CustomModelTrainer:
    """Handles Custom CNN model training and evaluation"""

    def __init__(self, config, model, data_generator):
        self.config = config
        self.model = model
        self.data_generator = data_generator
        self.history = None
        logging.info("CustomModelTrainer initialized")

    def train_model(self):
        """Train the Custom CNN model"""
        try:
            logging.info("Starting Custom CNN model training...")

            callbacks = self.model.get_callbacks()

            self.history = self.model.model.fit(
                self.data_generator.train_generator,
                epochs=self.config.EPOCHS,
                validation_data=self.data_generator.validation_generator,
                callbacks=callbacks,
                verbose=1
            )

            logging.info("Custom CNN model training completed successfully!")
            return True
        except Exception as e:
            logging.error(f"Error during Custom CNN training: {str(e)}")
            raise

    def generate_report(self):
        """Generate comprehensive validation report"""
        try:
            logging.info("Generating Custom CNN validation report...")

            # Get predictions
            validation_generator = self.data_generator.validation_generator
            y_pred = self.model.model.predict(validation_generator)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = validation_generator.classes

            # Calculate metrics
            accuracy = accuracy_score(y_true, y_pred_classes)
            precision, recall, f1, _ = precision_recall_fscore_support(
                y_true, y_pred_classes, average='weighted'
            )

            # Confusion matrix
            cm = confusion_matrix(y_true, y_pred_classes)
            FP = cm.sum(axis=0) - np.diag(cm)
            FN = cm.sum(axis=1) - np.diag(cm)
            TP = np.diag(cm)
            TN = cm.sum() - (FP + FN + TP)

            # Save report
            report_content = self._create_report_content(
                accuracy, precision, recall, f1, TP, TN, FP, FN, cm
            )

            with open(self.config.REPORT_PATH, 'w') as f:
                f.write(report_content)

            logging.info(f"Custom CNN validation report saved to: {self.config.REPORT_PATH}")
            
            # Also save JSON report
            self._save_json_report(accuracy, precision, recall, f1, TP, TN, FP, FN)
            
            return report_content

        except Exception as e:
            logging.error(f"Error generating Custom CNN validation report: {str(e)}")
            raise

    def _create_report_content(self, accuracy, precision, recall, f1, TP, TN, FP, FN, cm):
        """Create formatted report content"""
        report = []
        report.append("=" * 60)
        report.append("CUSTOM CNN MODEL - VALIDATION REPORT")
        report.append("=" * 60)
        report.append(f"Generated on: {datetime.datetime.now()}")
        report.append(f"Model: Custom CNN (Built from Scratch)")
        report.append(f"Dataset: Food34 (200 per class)")
        report.append("")

        # Overall metrics
        report.append("OVERALL METRICS:")
        report.append("-" * 30)
        report.append(f"Accuracy:  {accuracy:.4f}")
        report.append(f"Precision: {precision:.4f}")
        report.append(f"Recall:    {recall:.4f}")
        report.append(f"F1-Score:  {f1:.4f}")
        report.append("")

        # Per-class metrics
        report.append("PER-CLASS METRICS:")
        report.append("-" * 30)
        report.append(f"{'Class':<20} {'TP':<6} {'TN':<6} {'FP':<6} {'FN':<6}")
        report.append("-" * 60)

        class_names = list(self.data_generator.class_indices.keys())
        for i, class_name in enumerate(class_names):
            report.append(f"{class_name:<20} {TP[i]:<6} {TN[i]:<6} {FP[i]:<6} {FN[i]:<6}")

        report.append("")
        report.append("CONFUSION MATRIX:")
        report.append("-" * 30)
        report.append(str(cm))

        return "\n".join(report)

    def _save_json_report(self, accuracy, precision, recall, f1, TP, TN, FP, FN):
        """Save JSON report"""
        json_report_path = self.config.REPORT_PATH.replace('.txt', '.json')
        metrics = {
            "generated_on": str(datetime.datetime.now()),
            "model": "Custom_CNN",
            "accuracy": float(accuracy),
            "precision": float(precision),
            "recall": float(recall),
            "f1_score": float(f1),
            "TP": TP.tolist(),
            "TN": TN.tolist(),
            "FP": FP.tolist(),
            "FN": FN.tolist(),
            "class_indices": self.data_generator.class_indices
        }
        
        with open(json_report_path, 'w') as f:
            json.dump(metrics, f, indent=4)
        logging.info(f"Custom CNN JSON report saved to: {json_report_path}")

# **Main Pipeline Class**
class CustomFoodClassificationPipeline:
    def __init__(self):
        self.config = CustomModelConfig()
        self.data_generator = None
        self.model = None
        self.trainer = None

    def run_pipeline(self):
        """Execute the complete training pipeline"""
        try:
            logging.info("Starting Custom CNN Food Classification Pipeline on Kaggle...")

            # Step 1: Create data generators
            self.data_generator = CustomDataGenerator(self.config)
            self.data_generator.create_data_generators()

            # Step 2: Build and compile model
            self.model = CustomFoodModel(self.config, self.config.NUM_CLASSES)
            self.model.build_model()
            self.model.compile_model()

            # Display model summary
            logging.info("Custom CNN Model Summary:")
            self.model.display_summary()

            # Step 3: Train model
            self.trainer = CustomModelTrainer(self.config, self.model, self.data_generator)
            self.trainer.train_model()

            # Step 4: Generate report
            report = self.trainer.generate_report()

            # Final success message
            logging.info("Custom CNN Pipeline completed successfully!")
            print("\n" + "=" * 60)
            print("CUSTOM CNN TRAINING COMPLETED SUCCESSFULLY!")
            print("=" * 60)
            print(f"Model saved: {self.config.CUSTOM_MODEL_PATH}")
            print(f"Report saved: {self.config.REPORT_PATH}")
            print(f"Log file: {self.config.LOG_PATH}")

        except Exception as e:
            logging.error(f"Custom CNN Pipeline failed on Kaggle: {str(e)}")
            raise

# **Execute the pipeline**
if __name__ == "__main__":
    try:
        # Initialize config first
        config = CustomModelConfig()
        
        # Setup logging
        if setup_logging_custom(config):
            pipeline = CustomFoodClassificationPipeline()
            pipeline.run_pipeline()
        else:
            print("Failed to setup Custom CNN logging.")
    except Exception as e:
        print(f"Custom CNN main execution failed on Kaggle: {str(e)}")

# **Standalone Evaluation Code**
def evaluate_custom_model():
    """Standalone function to evaluate the trained Custom CNN model"""
    try:
        print("\n" + "="*50)
        print("CUSTOM CNN MODEL EVALUATION - KAGGLE")
        print("="*50)
        
        config = CustomModelConfig()
        
        # Load the trained model
        if os.path.exists(config.CUSTOM_MODEL_PATH):
            model = load_model(config.CUSTOM_MODEL_PATH)
            print("✓ Custom CNN model loaded successfully")
            
            # Create validation generator
            val_datagen = ImageDataGenerator(rescale=1./255)
            val_dir = os.path.join(config.DATA_DIR, 'val')
            
            val_generator = val_datagen.flow_from_directory(
                val_dir,
                target_size=config.IMG_SIZE,
                batch_size=config.BATCH_SIZE,
                class_mode='categorical',
                shuffle=False
            )

            # Predict
            y_pred = model.predict(val_generator)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_true = val_generator.classes

            # Calculate metrics
            accuracy = accuracy_score(y_true, y_pred_classes)
            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='weighted')
            
            print(f"✓ Custom CNN Evaluation Results:")
            print(f"  Accuracy:  {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall:    {recall:.4f}")
            print(f"  F1-Score:  {f1:.4f}")
            
            return True
        else:
            print("✗ Custom CNN model file not found in Kaggle working directory")
            return False
        
    except Exception as e:
        print(f"✗ Custom CNN evaluation failed: {str(e)}")
        return False

# Run evaluation after training
print("\nStarting Custom CNN evaluation on Kaggle...")
evaluate_custom_model()

# **ResNet Model Using CNN**

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import datetime

print("TensorFlow version:", tf.__version__)

class ResNetFoodClassifier:
    def __init__(self):
        self.data_dir = '/kaggle/input/food-classification-34-classes/food34_200_per_class'
        self.img_size = (224, 224)
        self.batch_size = 32
        self.epochs = 50
        self.num_classes = 34
        self.model_path = '/kaggle/working/resnet_food_model.h5'
        self.report_path = '/kaggle/working/ResNet_Model.txt'

    def load_data(self):
        """Load and prepare data"""
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            brightness_range=[0.8, 1.2],
            fill_mode='nearest'
        )

        val_datagen = ImageDataGenerator(rescale=1./255)

        train_dir = os.path.join(self.data_dir, 'train')
        val_dir = os.path.join(self.data_dir, 'val')

        train_gen = train_datagen.flow_from_directory(
            train_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=True
        )

        val_gen = val_datagen.flow_from_directory(
            val_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            shuffle=False
        )

        print(f"Found {train_gen.samples} training images")
        print(f"Found {val_gen.samples} validation images")
        return train_gen, val_gen, train_gen.class_indices

    def build_model(self):
        """Build ResNet50 model with ImageNet weights"""
        print("Building ResNet50 with ImageNet weights...")
        
        # Load pre-trained ResNet50
        base_model = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )

        # Freeze base model initially
        base_model.trainable = False

        # Add custom classification head
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.3)(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)
        
        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
        )
        
        print("ResNet50 model built successfully with ImageNet weights!")
        return model

    def train_model(self, train_generator, val_generator):
        """Train the model with two-phase approach"""
        model = self.build_model()

        callbacks = [
            EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
            ModelCheckpoint(self.model_path, monitor='val_accuracy', save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7)
        ]

        print("Phase 1: Training with frozen base layers...")
        history1 = model.fit(
            train_generator,
            epochs=25,
            validation_data=val_generator,
            callbacks=callbacks,
            verbose=1
        )

        print("Phase 2: Fine-tuning with unfrozen layers...")
        # Unfreeze last layers for fine-tuning
        for layer in model.layers[-20:]:
            if not isinstance(layer, BatchNormalization):
                layer.trainable = True

        # Recompile with lower learning rate
        model.compile(
            optimizer=Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
        )

        history2 = model.fit(
            train_generator,
            epochs=self.epochs,
            validation_data=val_generator,
            callbacks=callbacks,
            verbose=1,
            initial_epoch=25
        )
        
        return model, history2

    def calculate_detailed_metrics(self, y_true, y_pred, num_classes):
        """Calculate TP, TN, FP, FN for each class"""
        cm = confusion_matrix(y_true, y_pred)
        
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - TP
        FN = np.sum(cm, axis=1) - TP
        TN = np.sum(cm) - (FP + FN + TP)
        
        return TP, TN, FP, FN, cm

    def generate_report(self, model, val_generator, class_indices):
        """Generate comprehensive validation report with all metrics"""
        print("Generating detailed validation report...")
        
        # Get predictions
        y_pred = model.predict(val_generator)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = val_generator.classes
        
        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred_classes)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='weighted')
        
        # Calculate TP, TN, FP, FN
        TP, TN, FP, FN, cm = self.calculate_detailed_metrics(y_true, y_pred_classes, self.num_classes)
        
        # Generate report content
        report_content = f"""
RESNET50 MODEL - COMPREHENSIVE VALIDATION REPORT
============================================================
Generated: {datetime.datetime.now()}

DATASET INFORMATION:
- Training samples: {val_generator.samples}
- Number of classes: {self.num_classes}
- Class names: {list(class_indices.keys())}

OVERALL METRICS:
----------------------------------------
Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)
Precision: {precision:.4f} ({precision*100:.2f}%)
Recall:    {recall:.4f} ({recall*100:.2f}%)
F1-Score:  {f1:.4f} ({f1*100:.2f}%)

CONFUSION MATRIX SUMMARY:
----------------------------------------
Total True Positives (TP):  {int(np.sum(TP))}
Total True Negatives (TN):  {int(np.sum(TN))}
Total False Positives (FP): {int(np.sum(FP))}
Total False Negatives (FN): {int(np.sum(FN))}

DETAILED PER-CLASS METRICS:
----------------------------------------
{'Class':<20} {'TP':<8} {'TN':<8} {'FP':<8} {'FN':<8} {'Precision':<10} {'Recall':<10} {'F1-Score':<10}
{'-'*90}
"""
        # Add per-class metrics
        class_names = list(class_indices.keys())
        for i, class_name in enumerate(class_names):
            class_precision = TP[i] / (TP[i] + FP[i]) if (TP[i] + FP[i]) > 0 else 0
            class_recall = TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0
            class_f1 = 2 * (class_precision * class_recall) / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0
            
            report_content += f"{class_name:<20} {int(TP[i]):<8} {int(TN[i]):<8} {int(FP[i]):<8} {int(FN[i]):<8} {class_precision:.4f}     {class_recall:.4f}      {class_f1:.4f}\n"

        # Performance assessment
        report_content += f"""
PERFORMANCE ASSESSMENT:
----------------------------------------
"""
        if accuracy >= 0.90:
            report_content += "90%+ Accuracy Achieved!"
        elif accuracy >= 0.85:
            report_content += "85%+ Accuracy!"
        elif accuracy >= 0.80:
            report_content += "80%+ Accuracy!"
        elif accuracy >= 0.70:
            report_content += "70%+ Accuracy"
        else:
            report_content += "NEEDS IMPROVEMENT"

        report_content += f"""

MODEL TRAINING INFORMATION:
----------------------------------------
- Pre-trained model: ResNet50 with ImageNet weights
- Total epochs: {self.epochs}
- Batch size: {self.batch_size}
- Image size: {self.img_size}
- Number of classes: {self.num_classes}
- Model saved: {self.model_path}
"""

        # Save report
        with open(self.report_path, 'w') as f:
            f.write(report_content)

        print(f"Report saved to: {self.report_path}")
        return report_content

    def run(self):
        """Main execution function"""
        print("STARTING RESNET50 FOOD CLASSIFICATION")
        print("=" * 60)
        
        # Load data
        train_gen, val_gen, class_indices = self.load_data()
        
        # Train model
        model, history = self.train_model(train_gen, val_gen)
        print("Training completed!")
        
        # Generate report
        report_content = self.generate_report(model, val_gen, class_indices)
        
        # Final results summary
        print(f"\n📊 FINAL RESULTS SUMMARY:")
        print(f"   Accuracy:  {history.history['val_accuracy'][-1]:.4f} ({history.history['val_accuracy'][-1]*100:.2f}%)")
        print(f"   Precision: {history.history['val_precision'][-1]:.4f} ({history.history['val_precision'][-1]*100:.2f}%)")
        print(f"   Recall:    {history.history['val_recall'][-1]:.4f} ({history.history['val_recall'][-1]*100:.2f}%)")
        print(f"   Model saved: {self.model_path}")
        print(f"   Report saved: {self.report_path}")
        
        return model, report_content

# Run ResNet Model
print("Starting ResNet Model...")
resnet_classifier = ResNetFoodClassifier()
resnet_model, resnet_report = resnet_classifier.run()

TensorFlow version: 2.18.0
Starting ResNet Model...
STARTING RESNET50 FOOD CLASSIFICATION
Found 7956 images belonging to 34 classes.
Found 1372 images belonging to 34 classes.
Found 7956 training images
Found 1372 validation images
Building ResNet50 with ImageNet weights...
ResNet50 model built successfully with ImageNet weights!
Phase 1: Training with frozen base layers...
Epoch 1/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 212s 783ms/step - accuracy: 0.0301 - loss: 3.6383 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.0561 - val_loss: 3.4761 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 183s 736ms/step - accuracy: 0.0465 - loss: 3.4879 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.0576 - val_loss: 3.4380 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/25
249/249 ━━━━━━━━━━━━━━━━━━━━ 180s 721ms/step - accuracy: 0.0441 - loss: 3.4719 - precision: 0.0000e+00 -

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Data Augmentation process for 34 classes**

In [ ]:
# Check GPU and setup
!nvidia-smi

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

# Configure GPU for better performance
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✓ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")

# **Configuration and Imports**
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
import sys
import pickle
from PIL import Image
import random
import uuid
import shutil
from pathlib import Path

print("All imports completed successfully!")

# **Configuration Class**
class DataAugmentationConfig:
    """Configuration for data augmentation and file paths"""
    def __init__(self):
        # Kaggle dataset path (READ-ONLY)
        self.INPUT_DATA_DIR = '/kaggle/input/food-classification-34-classes/food34_200_per_class'
        # Working directory path (WRITABLE)
        self.WORKING_DATA_DIR = '/kaggle/working/food34_200_per_class'
        self.OUTPUT_DIR = '/kaggle/working'
        
        self.IMG_SIZE = (224, 224)
        self.TARGET_IMAGES_PER_CLASS = 200
        
        # Save paths
        self.AUGMENTATION_PICKLE_PATH = '/kaggle/working/data_augmentation.pkl'
        
        # Data augmentation parameters
        self.ROTATION_RANGE = 25
        self.WIDTH_SHIFT_RANGE = 0.15
        self.HEIGHT_SHIFT_RANGE = 0.15
        self.SHEAR_RANGE = 0.15
        self.ZOOM_RANGE = 0.15
        self.HORIZONTAL_FLIP = True
        self.BRIGHTNESS_RANGE = [0.8, 1.2]
        self.FILL_MODE = 'nearest'

# **Setup Logging**
def setup_augmentation_logging():
    """Setup logging for data augmentation process"""
    try:
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        
        # Clear existing handlers
        for handler in logger.handlers[:]:
            logger.removeHandler(handler)
            
        # Formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        
        # File handler
        file_handler = logging.FileHandler('/kaggle/working/data_augmentation.log', mode='w')
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        
        # Stream handler
        stream_handler = logging.StreamHandler(sys.stdout)
        stream_handler.setLevel(logging.INFO)
        stream_handler.setFormatter(formatter)
        
        # Add handlers
        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)
        
        logging.info("Data Augmentation Logging setup successfully for Kaggle!")
        return True
    except Exception as e:
        print(f"Logging setup failed: {e}")
        return False

# **Data Copy Class**
class DataCopier:
    """Copies data from read-only input to writable working directory"""
    
    def __init__(self, config):
        self.config = config
        logging.info("DataCopier initialized")
    
    def copy_dataset_to_working(self):
        """Copy entire dataset to working directory"""
        try:
            if os.path.exists(self.config.WORKING_DATA_DIR):
                logging.info("Working dataset already exists, skipping copy")
                return True
                
            logging.info("Copying dataset from input to working directory...")
            shutil.copytree(self.config.INPUT_DATA_DIR, self.config.WORKING_DATA_DIR)
            logging.info("✓ Dataset copied successfully to working directory")
            return True
            
        except Exception as e:
            logging.error(f"Error copying dataset: {str(e)}")
            raise

# **Data Analysis Class**
class DataAnalyzer:
    """Analyzes dataset and identifies classes needing augmentation"""
    
    def __init__(self, config):
        self.config = config
        self.train_counts = {}
        self.val_counts = {}
        self.test_counts = {}
        logging.info("DataAnalyzer initialized")
    
    def count_images_in_folder(self, folder_path):
        """Count images in a specific folder"""
        counts = {}
        if not os.path.exists(folder_path):
            return counts
            
        for class_name in sorted(os.listdir(folder_path)):
            class_path = os.path.join(folder_path, class_name)
            if os.path.isdir(class_path):
                image_count = sum(1 for f in os.listdir(class_path) 
                                if f.lower().endswith(('.jpg','.jpeg','.png','.bmp','.gif','.tif','.tiff')))
                counts[class_name] = image_count
        return counts
    
    def analyze_dataset(self):
        """Analyze complete dataset structure"""
        try:
            logging.info("Analyzing dataset structure...")
            
            # Count images in each split (using WORKING directory)
            train_dir = os.path.join(self.config.WORKING_DATA_DIR, 'train')
            val_dir = os.path.join(self.config.WORKING_DATA_DIR, 'val')
            test_dir = os.path.join(self.config.WORKING_DATA_DIR, 'test')
            
            self.train_counts = self.count_images_in_folder(train_dir)
            self.val_counts = self.count_images_in_folder(val_dir)
            self.test_counts = self.count_images_in_folder(test_dir)
            
            # Log results
            logging.info("=== DATASET ANALYSIS RESULTS ===")
            logging.info(f"Total classes found: {len(self.train_counts)}")
            
            logging.info("\nTRAINING SET (class: count):")
            for class_name, count in self.train_counts.items():
                logging.info(f"  {class_name}: {count} images")
            
            # Identify classes needing augmentation
            classes_needing_augmentation = [
                class_name for class_name, count in self.train_counts.items() 
                if count < self.config.TARGET_IMAGES_PER_CLASS
            ]
            
            logging.info(f"\nClasses needing augmentation: {len(classes_needing_augmentation)}")
            for class_name in classes_needing_augmentation:
                current_count = self.train_counts[class_name]
                needed = self.config.TARGET_IMAGES_PER_CLASS - current_count
                logging.info(f"  {class_name}: {current_count} -> needs {needed} more images")
            
            return classes_needing_augmentation
            
        except Exception as e:
            logging.error(f"Error analyzing dataset: {str(e)}")
            raise

# **Data Augmentation Class**
class DataAugmentor:
    """Handles data augmentation for classes with insufficient images"""
    
    def __init__(self, config):
        self.config = config
        self.augmentation_results = {}
        logging.info("DataAugmentor initialized")
    
    def setup_augmentation_generator(self):
        """Setup ImageDataGenerator for augmentation"""
        return ImageDataGenerator(
            rotation_range=self.config.ROTATION_RANGE,
            width_shift_range=self.config.WIDTH_SHIFT_RANGE,
            height_shift_range=self.config.HEIGHT_SHIFT_RANGE,
            shear_range=self.config.SHEAR_RANGE,
            zoom_range=self.config.ZOOM_RANGE,
            horizontal_flip=self.config.HORIZONTAL_FLIP,
            brightness_range=self.config.BRIGHTNESS_RANGE,
            fill_mode=self.config.FILL_MODE
        )
    
    def augment_class(self, class_folder, needed_count):
        """Augment images for a specific class"""
        try:
            # Get all image files in class folder
            image_files = [
                f for f in os.listdir(class_folder) 
                if f.lower().endswith(('.jpg','.jpeg','.png','.bmp','.gif','.tif','.tiff'))
            ]
            
            if not image_files:
                logging.warning(f"No source images found in {class_folder}")
                return 0
            
            datagen = self.setup_augmentation_generator()
            created_count = 0
            attempts = 0
            max_attempts = needed_count * 15  # Safety limit
            
            logging.info(f"Augmenting {class_folder}: need {needed_count} new images")
            
            while created_count < needed_count and attempts < max_attempts:
                attempts += 1
                
                # Randomly select source image
                source_filename = random.choice(image_files)
                source_path = os.path.join(class_folder, source_filename)
                
                try:
                    # Load and preprocess image
                    img = Image.open(source_path).convert('RGB')
                    img = img.resize(self.config.IMG_SIZE)
                    img_array = np.array(img)
                    
                    # Expand dimensions for generator
                    img_batch = np.expand_dims(img_array, axis=0)
                    
                    # Generate augmented image
                    aug_iterator = datagen.flow(img_batch, batch_size=1)
                    augmented_array = next(aug_iterator)[0].astype('uint8')
                    
                    # Convert back to PIL Image and save
                    augmented_img = Image.fromarray(augmented_array)
                    new_filename = f"aug_{uuid.uuid4().hex}.jpg"
                    new_filepath = os.path.join(class_folder, new_filename)
                    
                    augmented_img.save(new_filepath, quality=95, optimize=True)
                    created_count += 1
                    
                    if created_count % 20 == 0:  # Progress update
                        logging.info(f"  Created {created_count}/{needed_count} for {os.path.basename(class_folder)}")
                        
                except Exception as e:
                    logging.warning(f"Failed to augment {source_path}: {str(e)}")
                    continue
            
            logging.info(f"Completed {class_folder}: created {created_count} new images")
            return created_count
            
        except Exception as e:
            logging.error(f"Error augmenting class {class_folder}: {str(e)}")
            return 0
    
    def perform_augmentation(self, classes_to_augment, analyzer):
        """Perform augmentation for all specified classes"""
        try:
            logging.info("Starting data augmentation process...")
            
            train_dir = os.path.join(self.config.WORKING_DATA_DIR, 'train')
            augmentation_summary = {}
            
            for class_name in classes_to_augment:
                class_folder = os.path.join(train_dir, class_name)
                
                if not os.path.exists(class_folder):
                    logging.warning(f"Class folder not found: {class_folder}")
                    continue
                
                current_count = analyzer.train_counts[class_name]
                needed_count = self.config.TARGET_IMAGES_PER_CLASS - current_count
                
                if needed_count <= 0:
                    logging.info(f"{class_name} already has sufficient images ({current_count})")
                    continue
                
                # Perform augmentation
                created_count = self.augment_class(class_folder, needed_count)
                
                # Record results
                augmentation_summary[class_name] = {
                    'before': current_count,
                    'created': created_count,
                    'after': current_count + created_count,
                    'success': created_count >= needed_count * 0.8  # 80% success threshold
                }
            
            self.augmentation_results = augmentation_summary
            self._save_augmentation_report()
            return augmentation_summary
            
        except Exception as e:
            logging.error(f"Error in augmentation process: {str(e)}")
            raise
    
    def _save_augmentation_report(self):
        """Save augmentation results to pickle file"""
        try:
            # Prepare data for pickle
            augmentation_data = {
                'config': {
                    'target_images_per_class': self.config.TARGET_IMAGES_PER_CLASS,
                    'augmentation_parameters': {
                        'rotation_range': self.config.ROTATION_RANGE,
                        'width_shift_range': self.config.WIDTH_SHIFT_RANGE,
                        'height_shift_range': self.config.HEIGHT_SHIFT_RANGE,
                        'shear_range': self.config.SHEAR_RANGE,
                        'zoom_range': self.config.ZOOM_RANGE,
                        'horizontal_flip': self.config.HORIZONTAL_FLIP,
                        'brightness_range': self.config.BRIGHTNESS_RANGE
                    }
                },
                'results': self.augmentation_results,
                'timestamp': str(np.datetime64('now'))
            }
            
            # Save to pickle
            with open(self.config.AUGMENTATION_PICKLE_PATH, 'wb') as f:
                pickle.dump(augmentation_data, f)
            
            logging.info(f"Augmentation report saved to: {self.config.AUGMENTATION_PICKLE_PATH}")
            
        except Exception as e:
            logging.error(f"Error saving augmentation report: {str(e)}")

# **Verification Class**
class DataVerifier:
    """Verifies all created files and data integrity"""
    
    def __init__(self, config):
        self.config = config
        logging.info("DataVerifier initialized")
    
    def verify_augmentation(self):
        """Verify augmentation results"""
        try:
            logging.info("Starting augmentation verification...")
            
            verification_results = {
                'augmentation_pickle': self._verify_augmentation_pickle(),
                'image_counts': self._verify_image_counts()
            }
            
            self._log_verification_results(verification_results)
            return verification_results
            
        except Exception as e:
            logging.error(f"Verification failed: {str(e)}")
            raise
    
    def _verify_augmentation_pickle(self):
        """Verify augmentation pickle file"""
        try:
            if not os.path.exists(self.config.AUGMENTATION_PICKLE_PATH):
                return False, "Augmentation pickle file not found"
            
            with open(self.config.AUGMENTATION_PICKLE_PATH, 'rb') as f:
                data = pickle.load(f)
            
            required_keys = ['config', 'results', 'timestamp']
            if all(key in data for key in required_keys):
                return True, f"Augmentation pickle verified: {len(data['results'])} classes"
            else:
                return False, "Augmentation pickle missing required keys"
                
        except Exception as e:
            return False, f"Error loading augmentation pickle: {str(e)}"
    
    def _verify_image_counts(self):
        """Verify image counts after augmentation"""
        try:
            train_dir = os.path.join(self.config.WORKING_DATA_DIR, 'train')
            classes_with_insufficient = []
            
            for class_name in os.listdir(train_dir):
                class_path = os.path.join(train_dir, class_name)
                if os.path.isdir(class_path):
                    image_count = len([f for f in os.listdir(class_path) 
                                     if f.lower().endswith(('.jpg','.jpeg','.png'))])
                    if image_count < self.config.TARGET_IMAGES_PER_CLASS:
                        classes_with_insufficient.append((class_name, image_count))
            
            if not classes_with_insufficient:
                return True, f"All classes have at least {self.config.TARGET_IMAGES_PER_CLASS} images"
            else:
                return False, f"Classes with insufficient images: {classes_with_insufficient}"
                
        except Exception as e:
            return False, f"Error verifying image counts: {str(e)}"
    
    def _log_verification_results(self, results):
        """Log verification results"""
        logging.info("=== AUGMENTATION VERIFICATION RESULTS ===")
        for check_name, (status, message) in results.items():
            status_symbol = "✓" if status else "✗"
            logging.info(f"{status_symbol} {check_name}: {message}")

# **Main Pipeline Class**
class DataAugmentationPipeline:
    """Main pipeline for data augmentation only"""
    
    def __init__(self):
        self.config = DataAugmentationConfig()
        self.copier = None
        self.analyzer = None
        self.augmentor = None
        self.verifier = None
        logging.info("DataAugmentationPipeline initialized")
    
    def run_augmentation_pipeline(self):
        """Run the data augmentation pipeline"""
        try:
            logging.info("Starting Data Augmentation Pipeline on Kaggle...")
            
            # Step 1: Copy data to working directory
            self.copier = DataCopier(self.config)
            self.copier.copy_dataset_to_working()
            
            # Step 2: Analyze dataset
            self.analyzer = DataAnalyzer(self.config)
            classes_needing_augmentation = self.analyzer.analyze_dataset()
            
            # Step 3: Perform data augmentation
            self.augmentor = DataAugmentor(self.config)
            augmentation_results = self.augmentor.perform_augmentation(
                classes_needing_augmentation, self.analyzer
            )
            
            # Step 4: Verify augmentation
            self.verifier = DataVerifier(self.config)
            verification_results = self.verifier.verify_augmentation()
            
            # Final report
            self._generate_final_report(augmentation_results, verification_results)
            
            logging.info("Data Augmentation Pipeline completed successfully!")
            return True
            
        except Exception as e:
            logging.error(f"Augmentation pipeline failed: {str(e)}")
            raise
    
    def _generate_final_report(self, augmentation_results, verification_results):
        """Generate final summary report"""
        logging.info("\n" + "="*80)
        logging.info("DATA AUGMENTATION PIPELINE - FINAL REPORT")
        logging.info("="*80)
        
        # Augmentation summary
        if augmentation_results:
            total_created = sum(result['created'] for result in augmentation_results.values())
            successful_augmentations = sum(1 for result in augmentation_results.values() if result['success'])
            
            logging.info(f"AUGMENTATION SUMMARY:")
            logging.info(f"  Classes augmented: {len(augmentation_results)}")
            logging.info(f"  Total images created: {total_created}")
            logging.info(f"  Successful augmentations: {successful_augmentations}/{len(augmentation_results)}")
        
        # Verification summary
        all_checks_passed = all(status for status, _ in verification_results.values())
        
        logging.info(f"VERIFICATION SUMMARY:")
        for check_name, (status, message) in verification_results.items():
            status_text = "PASS" if status else "FAIL"
            logging.info(f"  {check_name}: {status_text} - {message}")
        
        logging.info(f"OVERALL STATUS: {'COMPLETED SUCCESSFULLY' if all_checks_passed else 'COMPLETED WITH ISSUES'}")
        logging.info("="*80)

# **Execute the augmentation pipeline**
if __name__ == "__main__":
    try:
        # Setup logging
        if setup_augmentation_logging():
            # Run the augmentation pipeline
            pipeline = DataAugmentationPipeline()
            success = pipeline.run_augmentation_pipeline()
            
            if success:
                print("\n" + "="*60)
                print("🎉 DATA AUGMENTATION COMPLETED SUCCESSFULLY!")
                print("="*60)
                print("Created files in /kaggle/working/:")
                print("  ✓ food34_200_per_class/ - Augmented dataset")
                print("  ✓ data_augmentation.pkl - Augmentation report")
                print("  ✓ data_augmentation.log - Detailed process log")
                print("\nDataset is ready for model training!")
            else:
                print("❌ Data augmentation failed. Check logs for details.")
        else:
            print("Failed to setup logging.")
            
    except Exception as e:
        print(f"Augmentation execution failed: {str(e)}")
        import traceback
        traceback.print_exc()

# **Quick verification function**
def quick_augmentation_verification():
    """Quick verification of augmentation results"""
    print("\n" + "="*50)
    print("AUGMENTATION VERIFICATION")
    print("="*50)
    
    config = DataAugmentationConfig()
    
    # Check augmentation pickle
    aug_exists = os.path.exists(config.AUGMENTATION_PICKLE_PATH)
    print(f"Augmentation pickle: {'✓ EXISTS' if aug_exists else '✗ MISSING'}")
    
    # Check working dataset
    working_exists = os.path.exists(config.WORKING_DATA_DIR)
    print(f"Working dataset: {'✓ EXISTS' if working_exists else '✗ MISSING'}")
    
    # Check log file
    log_exists = os.path.exists('/kaggle/working/data_augmentation.log')
    print(f"Log file: {'✓ EXISTS' if log_exists else '✗ MISSING'}")
    
    print("="*50)

# Run quick verification
quick_augmentation_verification()

# **JSON Files Creation Code for 34 Classes**

In [ ]:
# JSON Files Creation Code
import os
import json
import pickle
import logging
import sys

# **Configuration Class**
class JSONConfig:
    """Configuration for JSON file creation"""
    def __init__(self):
        self.OUTPUT_DIR = '/kaggle/working'
        self.JSON_PICKLE_PATH = '/kaggle/working/json_all_food_classes.pkl'
        self.JSON_FOLDER_PATH = '/kaggle/working/json_folder'

# **Setup Logging**
def setup_json_logging():
    """Setup logging for JSON creation process"""
    try:
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        
        # Clear existing handlers
        for handler in logger.handlers[:]:
            logger.removeHandler(handler)
            
        # Formatter
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        
        # File handler
        file_handler = logging.FileHandler('/kaggle/working/json_creation.log', mode='w')
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        
        # Stream handler
        stream_handler = logging.StreamHandler(sys.stdout)
        stream_handler.setLevel(logging.INFO)
        stream_handler.setFormatter(formatter)
        
        # Add handlers
        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)
        
        logging.info("JSON Creation Logging setup successfully for Kaggle!")
        return True
    except Exception as e:
        print(f"JSON Logging setup failed: {e}")
        return False

# **JSON File Creator Class**
class JSONCreator:
    """Creates JSON files for all 34 food classes"""
    
    def __init__(self, config):
        self.config = config
        self.classes_data = []
        logging.info("JSONCreator initialized")
    
    def create_classes_data(self):
        """Create the complete classes data structure"""
        self.classes_data = [
            {"class_name":"Baked_potato","label_index":0,"nutrition":{"protein":7,"fiber":5,"calories":260,"carbohydrates":60,"fat":1}},
            {"class_name":"Crispy_chicken","label_index":1,"nutrition":{"protein":28,"fiber":1,"calories":420,"carbohydrates":20,"fat":24}},
            {"class_name":"Donut","label_index":2,"nutrition":{"protein":4,"fiber":1,"calories":260,"carbohydrates":30,"fat":14}},
            {"class_name":"Fries","label_index":3,"nutrition":{"protein":4,"fiber":3,"calories":312,"carbohydrates":41,"fat":15}},
            {"class_name":"Hot_Dog","label_index":4,"nutrition":{"protein":10,"fiber":1,"calories":290,"carbohydrates":20,"fat":20}},
            {"class_name":"Sandwich","label_index":5,"nutrition":{"protein":18,"fiber":3,"calories":350,"carbohydrates":35,"fat":12}},
            {"class_name":"Taco","label_index":6,"nutrition":{"protein":12,"fiber":3,"calories":170,"carbohydrates":15,"fat":8}},
            {"class_name":"Taquito","label_index":7,"nutrition":{"protein":10,"fiber":2,"calories":220,"carbohydrates":18,"fat":11}},
            {"class_name":"apple_pie","label_index":8,"nutrition":{"protein":2,"fiber":2,"calories":320,"carbohydrates":45,"fat":15}},
            {"class_name":"burger","label_index":9,"nutrition":{"protein":25,"fiber":2,"calories":500,"carbohydrates":40,"fat":27}},
            {"class_name":"butter_naan","label_index":10,"nutrition":{"protein":8,"fiber":2,"calories":300,"carbohydrates":45,"fat":11}},
            {"class_name":"chai","label_index":11,"nutrition":{"protein":2,"fiber":0,"calories":120,"carbohydrates":15,"fat":5}},
            {"class_name":"chapati","label_index":12,"nutrition":{"protein":6,"fiber":3,"calories":120,"carbohydrates":20,"fat":3}},
            {"class_name":"cheesecake","label_index":13,"nutrition":{"protein":6,"fiber":0,"calories":350,"carbohydrates":25,"fat":26}},
            {"class_name":"chicken_curry","label_index":14,"nutrition":{"protein":22,"fiber":2,"calories":320,"carbohydrates":8,"fat":20}},
            {"class_name":"chole_bhature","label_index":15,"nutrition":{"protein":15,"fiber":8,"calories":650,"carbohydrates":80,"fat":30}},
            {"class_name":"dal_makhani","label_index":16,"nutrition":{"protein":12,"fiber":6,"calories":280,"carbohydrates":30,"fat":12}},
            {"class_name":"dhokla","label_index":17,"nutrition":{"protein":6,"fiber":2,"calories":120,"carbohydrates":18,"fat":4}},
            {"class_name":"fried_rice","label_index":18,"nutrition":{"protein":8,"fiber":2,"calories":350,"carbohydrates":55,"fat":12}},
            {"class_name":"ice_cream","label_index":19,"nutrition":{"protein":4,"fiber":0,"calories":207,"carbohydrates":24,"fat":11}},
            {"class_name":"idli","label_index":20,"nutrition":{"protein":6,"fiber":1,"calories":130,"carbohydrates":25,"fat":1}},
            {"class_name":"jalebi","label_index":21,"nutrition":{"protein":1,"fiber":0,"calories":300,"carbohydrates":50,"fat":12}},
            {"class_name":"kadai_paneer","label_index":22,"nutrition":{"protein":18,"fiber":3,"calories":420,"carbohydrates":12,"fat":30}},
            {"class_name":"kathi_rolls","label_index":23,"nutrition":{"protein":20,"fiber":3,"calories":420,"carbohydrates":40,"fat":18}},
            {"class_name":"kulfi","label_index":24,"nutrition":{"protein":6,"fiber":0,"calories":230,"carbohydrates":25,"fat":12}},
            {"class_name":"masala_dosa","label_index":25,"nutrition":{"protein":6,"fiber":3,"calories":200,"carbohydrates":30,"fat":6}},
            {"class_name":"momos","label_index":26,"nutrition":{"protein":10,"fiber":2,"calories":220,"carbohydrates":28,"fat":8}},
            {"class_name":"omlette","label_index":27,"nutrition":{"protein":12,"fiber":0,"calories":154,"carbohydrates":1,"fat":11}},
            {"class_name":"paani_puri","label_index":28,"nutrition":{"protein":4,"fiber":2,"calories":80,"carbohydrates":12,"fat":3}},
            {"class_name":"pakode","label_index":29,"nutrition":{"protein":6,"fiber":2,"calories":280,"carbohydrates":20,"fat":18}},
            {"class_name":"pav_bhaji","label_index":30,"nutrition":{"protein":9,"fiber":5,"calories":350,"carbohydrates":50,"fat":10}},
            {"class_name":"pizza","label_index":31,"nutrition":{"protein":12,"fiber":2,"calories":285,"carbohydrates":36,"fat":10}},
            {"class_name":"samosa","label_index":32,"nutrition":{"protein":5,"fiber":2,"calories":260,"carbohydrates":30,"fat":14}},
            {"class_name":"sushi","label_index":33,"nutrition":{"protein":9,"fiber":1,"calories":200,"carbohydrates":28,"fat":6}}
        ]
        
        logging.info(f"Created data for {len(self.classes_data)} classes")
        return self.classes_data
    
    def create_individual_json_files(self):
        """Create individual JSON files for each class"""
        try:
            # Create JSON folder
            os.makedirs(self.config.JSON_FOLDER_PATH, exist_ok=True)
            
            logging.info("Creating individual JSON files...")
            
            for class_data in self.classes_data:
                filename = f"{class_data['class_name']}.json"
                filepath = os.path.join(self.config.JSON_FOLDER_PATH, filename)
                
                with open(filepath, 'w') as f:
                    json.dump(class_data, f, indent=2)
                
                logging.info(f"Created: {filepath}")
            
            logging.info(f"All {len(self.classes_data)} JSON files created successfully!")
            return True
            
        except Exception as e:
            logging.error(f"Error creating JSON files: {str(e)}")
            raise
    
    def create_combined_pickle(self):
        """Create combined pickle file with all classes data"""
        try:
            with open(self.config.JSON_PICKLE_PATH, 'wb') as f:
                pickle.dump(self.classes_data, f)
            
            logging.info(f"Combined pickle file saved: {self.config.JSON_PICKLE_PATH}")
            
            # Verify the pickle file
            with open(self.config.JSON_PICKLE_PATH, 'rb') as f:
                verified_data = pickle.load(f)
            
            logging.info(f"Pickle verification successful: {len(verified_data)} classes loaded")
            return True
            
        except Exception as e:
            logging.error(f"Error creating combined pickle: {str(e)}")
            raise

# **Verification Class**
class JSONVerifier:
    """Verifies JSON files creation"""
    
    def __init__(self, config):
        self.config = config
        logging.info("JSONVerifier initialized")
    
    def verify_json_files(self):
        """Verify JSON files creation"""
        try:
            logging.info("Starting JSON files verification...")
            
            verification_results = {
                'json_pickle': self._verify_json_pickle(),
                'json_files': self._verify_json_files()
            }
            
            self._log_verification_results(verification_results)
            return verification_results
            
        except Exception as e:
            logging.error(f"JSON verification failed: {str(e)}")
            raise
    
    def _verify_json_pickle(self):
        """Verify JSON pickle file"""
        try:
            if not os.path.exists(self.config.JSON_PICKLE_PATH):
                return False, "JSON pickle file not found"
            
            with open(self.config.JSON_PICKLE_PATH, 'rb') as f:
                data = pickle.load(f)
            
            if len(data) == 34 and all('class_name' in item for item in data):
                return True, f"JSON pickle verified: {len(data)} classes"
            else:
                return False, f"JSON pickle has incorrect format: {len(data)} classes"
                
        except Exception as e:
            return False, f"Error loading JSON pickle: {str(e)}"
    
    def _verify_json_files(self):
        """Verify individual JSON files"""
        try:
            if not os.path.exists(self.config.JSON_FOLDER_PATH):
                return False, "JSON folder not found"
            
            json_files = [f for f in os.listdir(self.config.JSON_FOLDER_PATH) if f.endswith('.json')]
            
            if len(json_files) == 34:
                return True, f"All 34 JSON files created successfully"
            else:
                return False, f"Expected 34 JSON files, found {len(json_files)}"
                
        except Exception as e:
            return False, f"Error verifying JSON files: {str(e)}"
    
    def _log_verification_results(self, results):
        """Log verification results"""
        logging.info("=== JSON FILES VERIFICATION RESULTS ===")
        for check_name, (status, message) in results.items():
            status_symbol = "✓" if status else "✗"
            logging.info(f"{status_symbol} {check_name}: {message}")

# **Main JSON Creation Pipeline**
class JSONCreationPipeline:
    """Main pipeline for JSON files creation"""
    
    def __init__(self):
        self.config = JSONConfig()
        self.json_creator = None
        self.verifier = None
        logging.info("JSONCreationPipeline initialized")
    
    def run_json_creation_pipeline(self):
        """Run the JSON files creation pipeline"""
        try:
            logging.info("Starting JSON Files Creation Pipeline on Kaggle...")
            
            # Step 1: Create JSON files
            self.json_creator = JSONCreator(self.config)
            self.json_creator.create_classes_data()
            self.json_creator.create_individual_json_files()
            self.json_creator.create_combined_pickle()
            
            # Step 2: Verify JSON files
            self.verifier = JSONVerifier(self.config)
            verification_results = self.verifier.verify_json_files()
            
            # Final report
            self._generate_final_report(verification_results)
            
            logging.info("JSON Files Creation Pipeline completed successfully!")
            return True
            
        except Exception as e:
            logging.error(f"JSON creation pipeline failed: {str(e)}")
            raise
    
    def _generate_final_report(self, verification_results):
        """Generate final summary report"""
        logging.info("\n" + "="*80)
        logging.info("JSON FILES CREATION - FINAL REPORT")
        logging.info("="*80)
        
        # Verification summary
        all_checks_passed = all(status for status, _ in verification_results.values())
        
        logging.info(f"CREATION SUMMARY:")
        for check_name, (status, message) in verification_results.items():
            status_text = "PASS" if status else "FAIL"
            logging.info(f"  {check_name}: {status_text} - {message}")
        
        logging.info(f"OVERALL STATUS: {'COMPLETED SUCCESSFULLY' if all_checks_passed else 'COMPLETED WITH ISSUES'}")
        logging.info("="*80)

# **Execute the JSON creation pipeline**
if __name__ == "__main__":
    try:
        # Setup logging
        if setup_json_logging():
            # Run the JSON creation pipeline
            pipeline = JSONCreationPipeline()
            success = pipeline.run_json_creation_pipeline()
            
            if success:
                print("\n" + "="*60)
                print("🎉 JSON FILES CREATION COMPLETED SUCCESSFULLY!")
                print("="*60)
                print("Created files in /kaggle/working/:")
                print("  ✓ json_all_food_classes.pkl - Combined class data")
                print("  ✓ json_folder/ - Individual JSON files for all 34 classes")
                print("  ✓ json_creation.log - Detailed process log")
                print("\nJSON files are ready for use!")
            else:
                print("❌ JSON files creation failed. Check logs for details.")
        else:
            print("Failed to setup logging.")
            
    except Exception as e:
        print(f"JSON creation execution failed: {str(e)}")
        import traceback
        traceback.print_exc()

# **Quick verification function**
def quick_json_verification():
    """Quick verification of JSON files"""
    print("\n" + "="*50)
    print("JSON FILES VERIFICATION")
    print("="*50)
    
    config = JSONConfig()
    
    # Check JSON pickle
    json_pickle_exists = os.path.exists(config.JSON_PICKLE_PATH)
    print(f"JSON pickle: {'✓ EXISTS' if json_pickle_exists else '✗ MISSING'}")
    
    # Check JSON folder
    json_folder_exists = os.path.exists(config.JSON_FOLDER_PATH)
    if json_folder_exists:
        json_files = len([f for f in os.listdir(config.JSON_FOLDER_PATH) if f.endswith('.json')])
        print(f"JSON files: ✓ {json_files}/34 created")
    else:
        print(f"JSON folder: ✗ MISSING")
    
    # Check log file
    log_exists = os.path.exists('/kaggle/working/json_creation.log')
    print(f"Log file: {'✓ EXISTS' if log_exists else '✗ MISSING'}")
    
    print("="*50)

# Run quick verification
quick_json_verification()